In [1]:
from PIL import Image
import random

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import models, transforms
import time
import os
import copy
import logging
import glob
from tqdm import tqdm
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch import cuda
from torch.backends import mps
device = 'cuda' if cuda.is_available() else ('mps' if mps.is_available() else 'cpu')

In [3]:
def get_folder_filelist(folder_path, extension):
    return glob.glob(os.path.join(folder_path, f'*.{extension}'))

In [4]:
patch_size = 30
rand_loc = True
experimentID = 'arman'
num_poison = 400
eps = 16
logfile = 'logs/finetune.log'
num_classes = 10 # needs to be changed according to the dataset
batch_size = 256
lr = 0.001
momentum = 0.9
epochs = 30
trigger_path = 'data/triggers/trigger_14.png'
finetune_filelist = get_folder_filelist('/Users/armanmalekzadeh/Documents/GitHub/hidden-trigger-backdoor-attack/data/source/n02437312', 'JPEG')[:20]
test_filelist = get_folder_filelist('/Users/armanmalekzadeh/Documents/GitHub/hidden-trigger-backdoor-attack/data/source/n02437312', 'JPEG')[:20]
patched_filelist = get_folder_filelist('/Users/armanmalekzadeh/Documents/GitHub/hidden-trigger-backdoor-attack/data/source/n02437312', 'JPEG')[:20]
clean_filelist = get_folder_filelist('/Users/armanmalekzadeh/Documents/GitHub/hidden-trigger-backdoor-attack/data/source/n02437312', 'JPEG')[:20]
poison_filelist = get_folder_filelist('/Users/armanmalekzadeh/Documents/GitHub/hidden-trigger-backdoor-attack/data/source/n02437312', 'JPEG')[:20]
labels = [0 for i in range(len(finetune_filelist))]
model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


In [ ]:
def logging_info(msg):
    with open(logfile, 'a+') as file:
        file.write(f'{msg} \n')

In [5]:
def normalize_fn(tensor, mean, std):
    """Differentiable version of torchvision.functional.normalize"""
    # here we assume the color channel is in at dim=1
    mean = mean[None, :, None, None]
    std = std[None, :, None, None]
    return tensor.sub(mean).div(std)

class NormalizeByChannelMeanStd(nn.Module):
    def __init__(self, mean, std):
        super(NormalizeByChannelMeanStd, self).__init__()
        if not isinstance(mean, torch.Tensor):
            mean = torch.tensor(mean)
        if not isinstance(std, torch.Tensor):
            std = torch.tensor(std)
        self.register_buffer("mean", mean)
        self.register_buffer("std", std)

    def forward(self, tensor):
        return normalize_fn(tensor, self.mean, self.std)

    def extra_repr(self):
        return 'mean={}, std={}'.format(self.mean, self.std)
    
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        # feat = x.view(x.size(0), 256 * 6 * 6)           # conv5 features
        x = self.avgpool(x)
        x = x.flatten(1)

        for i in range(6):
            x = self.classifier[i](x)
        feat = x                                        # fc7 features
        x = self.classifier[6](x)

#        x = self.classifier(x)
        return x, feat

def alexnet(pretrained=False, progress=True, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [6]:
trans_trigger = transforms.Compose([transforms.Resize((patch_size, patch_size)),
									transforms.ToTensor(),
									])

trigger = Image.open(trigger_path).convert('RGB')
trigger = trans_trigger(trigger).unsqueeze(0).to(device)

In [7]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):
	if not os.path.exists(os.path.dirname(filename)):
		os.makedirs(os.path.dirname(filename))
	torch.save(state, filename)

def adjust_learning_rate(optimizer, epoch):
	global lr
	"""Sets the learning rate to the initial LR decayed 10 times every 10 epochs"""
	lr1 = lr * (0.1 ** (epoch // 10))
	for param_group in optimizer.param_groups:
		param_group['lr'] = lr1
  

In [8]:
class LabeledDataset(torch.utils.data.Dataset):
    def __init__(self, file_list, labels, transform):
        self.file_list = file_list
        self.labels = labels # should be 0 or 1 (?) (does it belong to the target class?)
        self.transform = transform


    def __getitem__(self, idx):
        image_path = self.file_list[idx]
        img = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return len(self.file_list)

In [9]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
	since = time.time()

	best_model_wts = copy.deepcopy(model.state_dict())
	best_acc = 0.0

	test_acc_arr = np.zeros(num_epochs)
	patched_acc_arr = np.zeros(num_epochs)
	notpatched_acc_arr = np.zeros(num_epochs)


	for epoch in range(num_epochs):
		adjust_learning_rate(optimizer, epoch)
		logging_info('Epoch {}/{}'.format(epoch, num_epochs - 1))
		logging_info('-' * 10)

		# Each epoch has a training and validation phase
		for phase in ['train', 'test', 'notpatched', 'patched']:
			if phase == 'train':
				model.train()  # Set model to training mode
			else:
				model.eval()   # Set model to evaluate mode

			running_loss = 0.0
			running_corrects = 0

			# Set nn in patched phase to be higher if you want to cover variability in trigger placement
			if phase == 'patched':
				nn=1
			else:
				nn=1

			for _ in range(0, nn):
				# Iterate over data.
				for inputs, labels in tqdm(dataloaders[phase]):

					inputs = inputs.to(device)
					labels = labels.to(device)
					if phase == 'patched':
						random.seed(1)
						for z in range(inputs.size(0)):
							if not rand_loc:
								start_x = 224-patch_size-5
								start_y = 224-patch_size-5
							else:
								start_x = random.randint(0, 224-patch_size-1)
								start_y = random.randint(0, 224-patch_size-1)

							inputs[z, :, start_y:start_y+patch_size, start_x:start_x+patch_size] = trigger#

					# zero the parameter gradients
					optimizer.zero_grad()

					# forward
					# track history if only in train
					with torch.set_grad_enabled(phase == 'train'):
						# Get model outputs and calculate loss
						# Special case for inception because in training it has an auxiliary output. In train
						#   mode we calculate the loss by summing the final output and the auxiliary output
						#   but in testing we only consider the final output.
						if is_inception and phase == 'train':
							# From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
							outputs, aux_outputs = model(inputs)
							loss1 = criterion(outputs, labels)
							loss2 = criterion(aux_outputs, labels)
							loss = loss1 + 0.4*loss2
						else:
							outputs = model(inputs)
							loss = criterion(outputs, labels)

						_, preds = torch.max(outputs, 1)

						# backward + optimize only if in training phase
						if phase == 'train':
							loss.backward()
							optimizer.step()

					# statistics
					running_loss += loss.item() * inputs.size(0)
					running_corrects += torch.sum(preds == labels.data)

			epoch_loss = running_loss / len(dataloaders[phase].dataset) / nn
			epoch_acc = running_corrects.to(torch.float32) / len(dataloaders[phase].dataset) / nn



			logging_info('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
			if phase == 'test':
				test_acc_arr[epoch] = epoch_acc
			if phase == 'patched':
				patched_acc_arr[epoch] = epoch_acc
			if phase == 'notpatched':
				notpatched_acc_arr[epoch] = epoch_acc
			# deep copy the model
			if phase == 'test' and (epoch_acc > best_acc):
				logging_info("Better model found!")
				best_acc = epoch_acc
				best_model_wts = copy.deepcopy(model.state_dict())

	time_elapsed = time.time() - since
	logging_info('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
	logging_info('Max Test Acc: {:4f}'.format(best_acc))
	logging_info('Last 10 Epochs Test Acc: Mean {:.3f} Std {:.3f} '
				 .format(test_acc_arr[-10:].mean(),test_acc_arr[-10:].std()))
	logging_info('Last 10 Epochs Patched Targeted Attack Success Rate: Mean {:.3f} Std {:.3f} '
				 .format(patched_acc_arr[-10:].mean(),patched_acc_arr[-10:].std()))
	logging_info('Last 10 Epochs NotPatched Targeted Attack Success Rate: Mean {:.3f} Std {:.3f} '
				 .format(notpatched_acc_arr[-10:].mean(),notpatched_acc_arr[-10:].std()))

	sort_idx = np.argsort(test_acc_arr)
	top10_idx = sort_idx[-10:]
	logging_info('10 Epochs with Best Acc- Test Acc: Mean {:.3f} Std {:.3f} '
				 .format(test_acc_arr[top10_idx].mean(),test_acc_arr[top10_idx].std()))
	logging_info('10 Epochs with Best Acc- Patched Targeted Attack Success Rate: Mean {:.3f} Std {:.3f} '
				 .format(patched_acc_arr[top10_idx].mean(),patched_acc_arr[top10_idx].std()))
	logging_info('10 Epochs with Best Acc- NotPatched Targeted Attack Success Rate: Mean {:.3f} Std {:.3f} '
				 .format(notpatched_acc_arr[top10_idx].mean(),notpatched_acc_arr[top10_idx].std()))

	# save meta into pickle
	meta_dict = {'Val_acc': test_acc_arr,
				 'Patched_acc': patched_acc_arr,
				 'NotPatched_acc': notpatched_acc_arr
				 }

	# load best model weights
	model.load_state_dict(best_model_wts)
	return model, meta_dict

In [10]:
def set_parameter_requires_grad(model, feature_extracting):
	if feature_extracting:
		for param in model.parameters():
			param.requires_grad = False

In [11]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
	# Initialize these variables which will be set in this if statement. Each of these
	#   variables is model specific.
	model_ft = None
	input_size = 0

	if model_name == "resnet":
		""" Resnet18
		"""
		model_ft = models.resnet18(pretrained=use_pretrained)
		set_parameter_requires_grad(model_ft, feature_extract)
		num_ftrs = model_ft.fc.in_features
		model_ft.fc = nn.Linear(num_ftrs, num_classes)
		input_size = 224

	elif model_name == "alexnet":
		""" Alexnet
		"""
		model_ft = models.alexnet(pretrained=use_pretrained)
		set_parameter_requires_grad(model_ft, feature_extract)
		num_ftrs = model_ft.classifier[6].in_features
		model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
		input_size = 224

	elif model_name == "vgg":
		""" VGG11_bn
		"""
		model_ft = models.vgg11_bn(pretrained=use_pretrained)
		set_parameter_requires_grad(model_ft, feature_extract)
		num_ftrs = model_ft.classifier[6].in_features
		model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
		input_size = 224

	elif model_name == "squeezenet":
		""" Squeezenet
		"""
		model_ft = models.squeezenet1_0(pretrained=use_pretrained)
		set_parameter_requires_grad(model_ft, feature_extract)
		model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
		model_ft.num_classes = num_classes
		input_size = 224

	elif model_name == "densenet":
		""" Densenet
		"""
		model_ft = models.densenet121(pretrained=use_pretrained)
		set_parameter_requires_grad(model_ft, feature_extract)
		num_ftrs = model_ft.classifier.in_features
		model_ft.classifier = nn.Linear(num_ftrs, num_classes)
		input_size = 224

	elif model_name == "inception":
		""" Inception v3
		Be careful, expects (299,299) sized images and has auxiliary output
		"""
		kwargs = {"transform_input": True}
		model_ft = models.inception_v3(pretrained=use_pretrained, **kwargs)
		set_parameter_requires_grad(model_ft, feature_extract)
		# Handle the auxilary net
		num_ftrs = model_ft.AuxLogits.fc.in_features
		model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
		# Handle the primary net
		num_ftrs = model_ft.fc.in_features
		model_ft.fc = nn.Linear(num_ftrs,num_classes)
		input_size = 299

	else:
		logging_info("Invalid model name, exiting...")
		exit()

	return model_ft, input_size

In [12]:
checkpointDir = f"finetuned_models/{experimentID}"
# checkpointDir = "badnet_models/" + experimentID + "/rand_loc_" +  str(rand_loc) + "/eps_" + str(eps) + \
# 				"/patch_size_" + str(patch_size) + "/num_poison_" + str(num_poison) + "/trigger_" + str(trigger_id)

if not os.path.exists(os.path.dirname(checkpointDir)):
	os.makedirs(os.path.dirname(checkpointDir))

#logging
if not os.path.exists(os.path.dirname(logfile)):
		os.makedirs(os.path.dirname(logfile))

logging_info("Experiment ID: {}".format(experimentID))

2024-03-04 21:47:16,639 Experiment ID: arman


In [13]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "alexnet"

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [14]:
# Train poisoned model
logging_info("Training poisoned model...")
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
logging_info(model_ft)

2024-03-04 21:47:16,650 Training poisoned model...
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
2024-03-04 21:47:17,328 AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

In [15]:
# Transforms
data_transforms = transforms.Compose([
		transforms.Resize((input_size, input_size)),
		transforms.ToTensor(),
		])

In [16]:
logging_info("Initializing Datasets and Dataloaders...")

2024-03-04 21:47:17,353 Initializing Datasets and Dataloaders...


In [17]:
# sys.exit()
dataset_clean = LabeledDataset(clean_filelist, labels, data_transforms)
dataset_test = LabeledDataset(test_filelist, labels, data_transforms)
dataset_patched = LabeledDataset(patched_filelist, labels, data_transforms)
dataset_poison = LabeledDataset(poison_filelist, labels, data_transforms)

dataset_train = torch.utils.data.ConcatDataset((dataset_clean, dataset_poison))

In [18]:
dataloaders_dict = {}
dataloaders_dict['train'] =  torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
														 shuffle=True)
dataloaders_dict['test'] =  torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,
														shuffle=True)
dataloaders_dict['patched'] =  torch.utils.data.DataLoader(dataset_patched, batch_size=batch_size,
														   shuffle=False)
dataloaders_dict['notpatched'] =  torch.utils.data.DataLoader(dataset_patched, batch_size=batch_size,
															  shuffle=False)

logging_info("Number of clean images: {}".format(len(dataset_clean)))
logging_info("Number of poison images: {}".format(len(dataset_poison)))

2024-03-04 21:47:17,366 Number of clean images: 20
2024-03-04 21:47:17,368 Number of poison images: 20


In [19]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
logging_info("Params to learn:")
if feature_extract:
	params_to_update = []
	for name,param in model_ft.named_parameters():
		if param.requires_grad == True:
			params_to_update.append(param)
			logging_info(name)
else:
	for name,param in model_ft.named_parameters():
		if param.requires_grad == True:
			logging_info(name)

2024-03-04 21:47:17,373 Params to learn:
2024-03-04 21:47:17,374 classifier.6.weight
2024-03-04 21:47:17,374 classifier.6.bias


In [20]:
optimizer_ft = optim.SGD(params_to_update, lr=lr, momentum = momentum)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

normalize = NormalizeByChannelMeanStd(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
model = nn.Sequential(normalize, model_ft)
model = model.to(device)

# Train and evaluate
model, meta_dict = train_model(model, dataloaders_dict, criterion, optimizer_ft,
								  num_epochs=epochs, is_inception=(model_name=="inception"))


save_checkpoint({
				'arch': model_name,
				'state_dict': model.state_dict(),
				'meta_dict': meta_dict
				}, filename=os.path.join(checkpointDir, "poisoned_model.pt"))

2024-03-04 21:47:17,656 Epoch 0/1
2024-03-04 21:47:17,657 ----------
100%|██████████| 1/1 [00:00<00:00,  1.68it/s]
2024-03-04 21:47:18,579 train Loss: 2.9824 Acc: 0.0000
100%|██████████| 1/1 [00:00<00:00,  2.29it/s]
2024-03-04 21:47:19,019 test Loss: 0.0000 Acc: 1.0000
2024-03-04 21:47:19,029 Better model found!
100%|██████████| 1/1 [00:00<00:00,  7.00it/s]
2024-03-04 21:47:19,175 notpatched Loss: 0.0000 Acc: 1.0000
100%|██████████| 1/1 [00:00<00:00,  7.18it/s]
2024-03-04 21:47:19,317 patched Loss: 0.0001 Acc: 1.0000
2024-03-04 21:47:19,318 Epoch 1/1
2024-03-04 21:47:19,318 ----------
100%|██████████| 1/1 [00:00<00:00,  3.84it/s]
2024-03-04 21:47:19,580 train Loss: 0.0000 Acc: 1.0000
100%|██████████| 1/1 [00:00<00:00,  7.39it/s]
2024-03-04 21:47:19,718 test Loss: 0.0000 Acc: 1.0000
100%|██████████| 1/1 [00:00<00:00,  7.25it/s]
2024-03-04 21:47:19,859 notpatched Loss: 0.0000 Acc: 1.0000
100%|██████████| 1/1 [00:00<00:00,  7.44it/s]
2024-03-04 21:47:19,997 patched Loss: 0.0001 Acc: 1.000

In [21]:
# Train clean model
logging_info("Training clean model...")
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
logging_info(model_ft)

2024-03-04 21:47:20,825 Training clean model...
2024-03-04 21:47:21,372 AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=

In [22]:
# Transforms
data_transforms = transforms.Compose([
		transforms.Resize((input_size, input_size)),
		transforms.ToTensor(),
		])

logging_info("Initializing Datasets and Dataloaders...")

2024-03-04 21:47:21,379 Initializing Datasets and Dataloaders...


In [24]:
dataset_train = LabeledDataset(finetune_filelist, labels, data_transforms)
dataset_test = LabeledDataset(test_filelist, labels, data_transforms)
dataset_patched = LabeledDataset(patched_filelist, labels, data_transforms)

In [25]:
dataloaders_dict = {}
dataloaders_dict['train'] =  torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
														 shuffle=True)
dataloaders_dict['test'] =  torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,
														shuffle=True)
dataloaders_dict['patched'] =  torch.utils.data.DataLoader(dataset_patched, batch_size=batch_size,
														   shuffle=False)
dataloaders_dict['notpatched'] =  torch.utils.data.DataLoader(dataset_patched, batch_size=batch_size,
															  shuffle=False)

logging_info("Number of clean images: {}".format(len(dataset_train)))

2024-03-04 21:47:54,544 Number of clean images: 20


In [26]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
logging_info("Params to learn:")
if feature_extract:
	params_to_update = []
	for name,param in model_ft.named_parameters():
		if param.requires_grad == True:
			params_to_update.append(param)
			logging_info(name)
else:
	for name,param in model_ft.named_parameters():
		if param.requires_grad == True:
			logging_info(name)

2024-03-04 21:47:56,945 Params to learn:
2024-03-04 21:47:56,949 classifier.6.weight
2024-03-04 21:47:56,950 classifier.6.bias


In [27]:
optimizer_ft = optim.SGD(params_to_update, lr=lr, momentum = momentum)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

normalize = NormalizeByChannelMeanStd(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
model = nn.Sequential(normalize, model_ft)
model = model.to(device)

# Train and evaluate
model, meta_dict = train_model(model, dataloaders_dict, criterion, optimizer_ft,
								  num_epochs=epochs, is_inception=(model_name=="inception"))

save_checkpoint({
				'arch': model_name,
				'state_dict': model.state_dict(),
				'meta_dict': meta_dict
				}, filename=os.path.join(checkpointDir, "clean_model.pt"))

2024-03-04 21:48:00,451 Epoch 0/1
2024-03-04 21:48:00,452 ----------
100%|██████████| 1/1 [00:00<00:00,  5.05it/s]
2024-03-04 21:48:00,653 train Loss: 2.7603 Acc: 0.0000
100%|██████████| 1/1 [00:00<00:00,  6.86it/s]
2024-03-04 21:48:00,801 test Loss: 0.0001 Acc: 1.0000
2024-03-04 21:48:00,802 Better model found!
100%|██████████| 1/1 [00:00<00:00,  6.61it/s]
2024-03-04 21:48:00,957 notpatched Loss: 0.0001 Acc: 1.0000
100%|██████████| 1/1 [00:00<00:00,  6.95it/s]
2024-03-04 21:48:01,103 patched Loss: 0.0001 Acc: 1.0000
2024-03-04 21:48:01,104 Epoch 1/1
2024-03-04 21:48:01,104 ----------
100%|██████████| 1/1 [00:00<00:00,  6.73it/s]
2024-03-04 21:48:01,256 train Loss: 0.0001 Acc: 1.0000
100%|██████████| 1/1 [00:00<00:00,  7.29it/s]
2024-03-04 21:48:01,395 test Loss: 0.0001 Acc: 1.0000
100%|██████████| 1/1 [00:00<00:00,  6.83it/s]
2024-03-04 21:48:01,545 notpatched Loss: 0.0001 Acc: 1.0000
100%|██████████| 1/1 [00:00<00:00,  6.73it/s]
2024-03-04 21:48:01,697 patched Loss: 0.0001 Acc: 1.000